In [1]:
from enum import Enum

class ImmoWeltUrls(Enum):
    BUY_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/kaufen?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    # add price range to avoid "consulting"-offers without named price 
    BUY_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/kaufen?d=true&pma=10000000&pmi=10&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"



In [2]:
from apify_client import ApifyClient

def get_dataset_items(url, limit):
    # Initialize the ApifyClient with your API token
    client = ApifyClient("apify_api_ODNYdzGzgsEyN9NGAQ3vyPyqTuj1jq34zPry")

    # Prepare the Actor input
    run_input = {
        "startUrls": [{ "url": url }],
        "limit": limit,
        "proxyConfig": { "useApifyProxy": True },
    }

    # Run the Actor and wait for it to finish
    run = client.actor("b4d9dfcf9MzBnyW7z").call(run_input=run_input)

    # Get the output of the Actor run
    dataset_items = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        dataset_items.append(item)

    return dataset_items



In [3]:
# get data 
# flats_to_buy = get_dataset_items(ImmoWeltUrls.BUY_FLATS_WUE_10km.value, 50)
houses_to_buy = get_dataset_items(ImmoWeltUrls.BUY_HOUSES_WUE_10km.value, 50)
# flats_to_rent = get_dataset_items(ImmoWeltUrls.RENT_FLATS_WUE_10km.value, 50)
# houses_to_rent = get_dataset_items(ImmoWeltUrls.RENT_HOUSES_WUE_10km.value, 50)

In [4]:
import openpyxl
import json

def write_data_to_excel(data_objects, file_name):
    # Create a new workbook and select the active sheet
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Write the headers
    headers = list(data_objects[0].keys())
    for col_num, header in enumerate(headers, 1):
        sheet.cell(row=1, column=col_num).value = header

    # Write the data rows
    for row_num, data in enumerate(data_objects, 2):
        for col_num, header in enumerate(headers, 1):
            value = json.dumps(data.get(header, ""))  # Convert value to string, use empty string if key not found
            sheet.cell(row=row_num, column=col_num).value = value

    # Save the workbook
    workbook.save(file_name)

In [5]:
# write data to excel for each category
# write_data_to_excel(flats_to_buy, "flats_to_buy.xlsx")
write_data_to_excel(houses_to_buy, "houses_to_buy1.xlsx")
# write_data_to_excel(flats_to_rent, "flats_to_rent.xlsx")
# write_data_to_excel(houses_to_rent, "houses_to_rent.xlsx")
